In [140]:
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM 
from keras.layers import Embedding, Activation, Dense, Dropout, Input

In [64]:
data= pd.read_csv("C://Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/Toxic Comment Classification/train.csv")

In [65]:
data.shape

(159571, 8)

In [32]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [89]:
data.drop('id', axis=1, inplace=True)

In [91]:
toxic=data[data['toxic']==1]

In [92]:
toxic.shape

(15294, 7)

In [93]:
nontoxic=data[data['toxic']==0].head(15294)

In [94]:
nontoxic.shape

(15294, 7)

In [95]:
merge= np.r_[toxic,nontoxic]

In [97]:
type(merge)

numpy.ndarray

In [131]:
x= merge[:,0]
y=merge[:,1:7]

In [99]:
y[:5]

array([[1, 1, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 0]], dtype=object)

In [125]:
y= to_categorical(y)

In [101]:
y.shape

(30588, 6, 2)

In [118]:
y[:1]

array([[1, 1, 1, 0, 1, 0]], dtype=object)

In [132]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=4)

In [133]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(xtrain)
sequences = tok.texts_to_sequences(xtrain)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [121]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((21411,), (9177,), (21411, 6), (9177, 6))

In [107]:
sequences[:1]

[[411, 162, 2, 317, 17, 21, 167]]

In [134]:
def model():
    
    model = Sequential()
    model.add(Embedding(1000,50,input_length=max_len))
    model.add(LSTM(64))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [135]:
m= model()

m.fit(sequences_matrix,ytrain,epochs=10, validation_split=0.2)

Train on 17128 samples, validate on 4283 samples
Epoch 1/10
17128/17128 [==============================] - 65s 4ms/step - loss: 0.3806 - acc: 0.8249 - val_loss: 0.3717 - val_acc: 0.8341
Epoch 2/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3698 - acc: 0.8314 - val_loss: 0.3665 - val_acc: 0.8310
Epoch 3/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3653 - acc: 0.8331 - val_loss: 0.3661 - val_acc: 0.8340
Epoch 4/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3615 - acc: 0.8333 - val_loss: 0.3633 - val_acc: 0.8300
Epoch 5/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3588 - acc: 0.8339 - val_loss: 0.3642 - val_acc: 0.8236
Epoch 6/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3559 - acc: 0.8347 - val_loss: 0.3615 - val_acc: 0.8340
Epoch 7/10
17128/17128 [==============================] - 61s 4ms/step - loss: 0.3532 - acc: 0.8365 - val_loss: 0.3644 - val_acc:

In [143]:
test_sequences = tok.texts_to_sequences(xtest)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = m.evaluate(test_sequences_matrix,ytest)

In [144]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.372
  Accuracy: 0.825
